This code calculates characterization factors for geo units (large countries are divided into the largest administrative units within the country and countries including islands are divided into mainland and islands). The raster-values within a unit are averaged using the rasterstats package and its zonal_stats function whereby the values on non-crop area are set to no data.

In [1]:
import os
import mypackages.myrasters as mr
from osgeo import gdal, ogr, osr
from rasterstats import zonal_stats
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
data_dir = os.path.join('..', 'data')
cf_dir = os.path.join('..', 'output/characterization_factors')
area_dir = os.path.join('..', 'output/impact')
out_dir = os.path.join('..', 'output/characterization_factors')

In [3]:
shape = os.path.join(data_dir, 'shapefiles/geo_units.shp')

In [4]:
cf_raster = mr.MyRaster(os.path.join(cf_dir, 'ts_100yrs.tif'))

In [5]:
cf = cf_raster.get_array()

In [6]:
crop_area_raster = mr.MyRaster(os.path.join(area_dir, 'crop_area.tif'))

In [7]:
crop_area = crop_area_raster.get_array()

In [8]:
%%time

crop_area_resampled = mr.resample_array_to_higher_resolution(array=crop_area, resample_factor=10)

CPU times: user 14.7 s, sys: 6.86 s, total: 21.5 s
Wall time: 21.5 s


In [9]:
cf[crop_area_resampled == 0] = -1

In [10]:
out_filename = 'ts_100yrs_crop_area'
mr.array2geotiff_rastercopy(cf, os.path.join(out_dir, out_filename), cf_raster.raster)

In [11]:
raster = os.path.join(cf_dir, 'ts_100yrs_crop_area.tif')

In [12]:
%%time

stats = zonal_stats(shape, raster, stats='mean')

/home/sothomas/miniconda3/envs/py36_gis/lib/python3.6/site-packages/rasterstats/main.py:142: FutureWarning: The value of this property will change in version 1.0. Please see https://github.com/mapbox/rasterio/issues/86 for details.
  with Raster(raster, affine, nodata, band) as rast:
/home/sothomas/miniconda3/envs/py36_gis/lib/python3.6/site-packages/rasterstats/io.py:242: FutureWarning: GDAL-style transforms are deprecated and will not be supported in Rasterio 1.0.
  self.affine = guard_transform(self.src.transform)


CPU times: user 29 s, sys: 2.86 s, total: 31.9 s
Wall time: 31.9 s


In [13]:
ts_100yrs = [i['mean'] for i in stats]

In [14]:
cf_raster = mr.MyRaster(os.path.join(cf_dir, 'ms_100yrs.tif'))

In [15]:
cf = cf_raster.get_array()

In [16]:
cf[crop_area_resampled == 0] = -1

In [17]:
out_filename = 'ms_100yrs_crop_area'
mr.array2geotiff_rastercopy(cf, os.path.join(out_dir, out_filename), cf_raster.raster)

In [18]:
raster = os.path.join(cf_dir, 'ms_100yrs_crop_area.tif')

In [19]:
%%time

stats = zonal_stats(shape, raster, stats='mean')

/home/sothomas/miniconda3/envs/py36_gis/lib/python3.6/site-packages/rasterstats/main.py:142: FutureWarning: The value of this property will change in version 1.0. Please see https://github.com/mapbox/rasterio/issues/86 for details.
  with Raster(raster, affine, nodata, band) as rast:
/home/sothomas/miniconda3/envs/py36_gis/lib/python3.6/site-packages/rasterstats/io.py:242: FutureWarning: GDAL-style transforms are deprecated and will not be supported in Rasterio 1.0.
  self.affine = guard_transform(self.src.transform)


CPU times: user 28 s, sys: 1.6 s, total: 29.6 s
Wall time: 29.6 s


In [20]:
ms_100yrs = [i['mean'] for i in stats]

In [21]:
cf_raster = mr.MyRaster(os.path.join(cf_dir, 'bs_100yrs.tif'))

In [22]:
cf = cf_raster.get_array()

In [23]:
cf[crop_area_resampled == 0] = -1

In [24]:
out_filename = 'bs_100yrs_crop_area'
mr.array2geotiff_rastercopy(cf, os.path.join(out_dir, out_filename), cf_raster.raster)

In [ ]:
raster = os.path.join(cf_dir, 'bs_100yrs_crop_area.tif')

In [ ]:
%%time

stats = zonal_stats(shape, raster, stats='mean')

/home/sothomas/miniconda3/envs/py36_gis/lib/python3.6/site-packages/rasterstats/main.py:142: FutureWarning: The value of this property will change in version 1.0. Please see https://github.com/mapbox/rasterio/issues/86 for details.
  with Raster(raster, affine, nodata, band) as rast:
/home/sothomas/miniconda3/envs/py36_gis/lib/python3.6/site-packages/rasterstats/io.py:242: FutureWarning: GDAL-style transforms are deprecated and will not be supported in Rasterio 1.0.
  self.affine = guard_transform(self.src.transform)


In [ ]:
bs_100yrs = [i['mean'] for i in stats]

Create a geopandas dataframe and add the characterization factors to it:

In [ ]:
gdf = gpd.read_file(shape)

In [ ]:
gdf['ts_100yrs'] = ts_100yrs
gdf['ms_100yrs'] = ms_100yrs
gdf['bs_100yrs'] = bs_100yrs

In [ ]:
out_filename = 'geo_units_cf_crop_area.shp'
gdf.to_file(os.path.join(out_dir, out_filename))

In [ ]:
a = pd.DataFrame(gdf[['id_name', 'country', 'ts_100yrs', 'ms_100yrs', 'bs_100yrs']])

In [ ]:
out_filename = 'geo_units_cf_crop_area.xlsx'
a.to_excel(os.path.join(out_dir, out_filename))